In [118]:
import sqlite3
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
import datetime


In [113]:
# 建表
# 连接到 SQLite 数据库
# 如果文件不存在，会自动在当前目录创建一个名为 'langchain.db' 的数据库文件
conn = sqlite3.connect('langchain.db')

# 创建一个 Cursor 对象并通过它执行 SQL 语句
c = conn.cursor()
# 创建表
c.execute('''
create table if not exists schedules 
(
    id          INTEGER
        primary key autoincrement,
    start_time  TEXT default (strftime('%Y-%m-%d %H:%M:%S', 'now', 'localtime')) not null,
    description TEXT default ''                                                  not null
);
''')


# 提交事务
conn.commit()
# 关闭连接
conn.close()

print("数据库和表已成功创建！")

数据库和表已成功创建！


In [67]:
import sqlite3


def connect_db():
    """ 连接到数据库 """
    conn = sqlite3.connect('langchain.db')
    return conn
    
@tool
def add_schedule(start_time : str, description : str) -> str: 
    """ 新增日程，比如2024-05-03 20:00:00, 周会 """
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO schedules (start_time, description) VALUES (?, ?);
    """, (start_time, description,))
    conn.commit()
    conn.close()
    return "true"

@tool
def delete_schedule_by_time(start_time : str) -> str:
    """ 根据时间删除日程 """
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("""
        DELETE FROM schedules WHERE start_time = ?;
    """, (start_time,))
    conn.commit()
    conn.close()
    return "true"
    
@tool
def get_schedules_by_date(query_date : str) -> str:
    """ 根据日期查询日程，比如 获取2024-05-03的所有日程 """
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("""
        SELECT start_time, description FROM schedules WHERE start_time LIKE ?;
    """, (f"{query_date}%",))
    schedules = cursor.fetchall()
    conn.close()
    return str(schedules)

In [68]:
llm = ChatOpenAI(model="gpt-3.5-turbo", max_tokens=4096)
tools = [add_schedule, delete_schedule_by_time, get_schedules_by_date]
llm_with_tools = llm.bind_tools(tools)

In [69]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一个日程管理助手",
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input} \n\n 当前时间为：{current_time}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [70]:
agent = create_tool_calling_agent(llm_with_tools, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [78]:
def ask(question):
    res = agent_executor.invoke(
        {
            "input": question,
            "current_time": datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
    )
    return res["output"]

In [ ]:
while True:
    question = input(">")
    if question.lower() == '退出':
        break
    print(ask(question))

In [122]:
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [123]:

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [126]:
res = agent_with_chat_history.invoke(
        {
            "input": "我明天有啥日程？",
            "current_time": datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        },
        config={"configurable": {"session_id": "111"}}
    )
res

Parent run 0b63b0a3-8393-409b-b303-3d5e7eee5dbf not found for run c125bee1-d0d1-400d-83f0-1fd0e75f3ee0. Treating as a root run.


{'input': '我明天有啥日程？',
 'current_time': '2024-05-04 13:44:43',
 'chat_history': [],
 'output': '明天的日程安排如下：\n- 09:00:00: 沟通会'}

In [127]:
store

{'222': InMemoryChatMessageHistory(messages=[HumanMessage(content='你好？'), AIMessage(content='你好！有什么可以帮助你的吗？'), HumanMessage(content='我明天有啥日程？'), AIMessage(content='明天的日程安排如下：\n1. 时间：2024-05-05 09:00:00，日程：沟通会')]),
 '111': InMemoryChatMessageHistory(messages=[HumanMessage(content='我明天有啥日程？'), AIMessage(content='明天的日程安排如下：\n- 09:00:00: 沟通会')])}

In [105]:
message_history

InMemoryChatMessageHistory(messages=[HumanMessage(content='你好'), AIMessage(content='你好！有什么可以帮助你的吗？'), HumanMessage(content='讲个笑话'), AIMessage(content='抱歉，我不会讲笑话。不过如果你有其他需要的话，欢迎告诉我哦！'), HumanMessage(content='我今天有啥安排？'), AIMessage(content='今天的日程安排如下：\n1. 10:00:00 - 技术分享会\n2. 15:00:00 - 周会\n\n希望你能顺利完成今天的安排！如果有其他问题或需要帮助，请随时告诉我。'), HumanMessage(content='明天早上9点，我要开一个沟通会'), AIMessage(content='好的，我已经帮你添加了明天早上9点的沟通会日程。希望会议顺利进行！如果还有其他需求或问题，请随时告诉我。'), HumanMessage(content='我今天下午有会吗？'), AIMessage(content='今天下午您有一场周会安排在15:00。希望您能准时参加！如果有其他问题或需要帮助，请随时告诉我。'), HumanMessage(content='我今天下午有会吗？'), AIMessage(content='根据我的查询，今天下午您有一场周会安排在15:00。希望您能准时参加！如果有其他问题或需要帮助，请随时告诉我。')])